In [1]:
import torch

In [2]:
from torch import nn
from torch.utils.data import DataLoader

In [4]:
import torchvision

In [6]:
from torchvision import datasets
from torchvision.transforms import ToTensor

In [13]:
training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor(),
    )

In [9]:
test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor(),
)

In [10]:
test_data

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [11]:
batch_size = 64

In [14]:
train_dataloader = DataLoader(
    training_data, 
    batch_size = batch_size,
    )

In [15]:
test_dataloader = DataLoader(
    test_data,
    batch_size = batch_size,
)

In [17]:
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]:{X.shape}")
    print(f"Shape of y: {y.shape}, {y.dtype}")
    break

Shape of X [N, C, H, W]:torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]), torch.int64


In [19]:
device = "cuda" if torch.cuda.is_available() else "gpu"

In [21]:
print(f"Using {device} device")

Using cuda device


In [36]:
 class NeuralNetwork(nn.Module):
        def __init__(self):
            super(NeuralNetwork, self).__init__()
            self.flatten = nn.Flatten()
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(28*28, 512),
                nn.ReLU(),
                nn.Linear(512, 512),
                nn.ReLU(),
                nn.Linear(512, 10)
                )
        
        def forward(self, x):
            x = self.flatten(x)
            logits = self.linear_relu_stack(x)
            return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [37]:
loss_fn = nn.CrossEntropyLoss()

In [38]:
optimizer = torch.optim.SGD(
    model.parameters(), 
    lr = 1e-3,
)

In [39]:
def train(
    dataloader,
    model,
    loss_fn,
    optimizer,
    ):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}]/{size:>5d}")

In [47]:
def test(
    dataloader,
    model,
    loss_fn,
    ):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        print(f"Test Error:\n Accuracy {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")

In [48]:
epochs = 5

for t in range(epochs):
    print(f"\nEpoch {t+1}\n---------------------")
    train(
        train_dataloader,
        model, 
        loss_fn,
        optimizer,
        )
    test(test_dataloader, model, loss_fn)


Epoch 1
---------------------
loss: 1.176709 [    0]/60000
loss: 1.167671 [ 6400]/60000
loss: 0.990554 [12800]/60000
loss: 1.120201 [19200]/60000
loss: 0.997473 [25600]/60000
loss: 1.026986 [32000]/60000
loss: 1.061035 [38400]/60000
loss: 1.006832 [44800]/60000
loss: 1.042521 [51200]/60000
loss: 0.970586 [57600]/60000
Test Error:
 Accuracy 66.2%, Avg loss: 0.988763

Epoch 2
---------------------
loss: 1.055813 [    0]/60000
loss: 1.070935 [ 6400]/60000
loss: 0.875245 [12800]/60000
loss: 1.028220 [19200]/60000
loss: 0.907668 [25600]/60000
loss: 0.932026 [32000]/60000
loss: 0.983682 [38400]/60000
loss: 0.931080 [44800]/60000
loss: 0.962937 [51200]/60000
loss: 0.903690 [57600]/60000
Test Error:
 Accuracy 67.3%, Avg loss: 0.915299

Epoch 3
---------------------
loss: 0.965604 [    0]/60000
loss: 1.003141 [ 6400]/60000
loss: 0.791658 [12800]/60000
loss: 0.962074 [19200]/60000
loss: 0.846889 [25600]/60000
loss: 0.861435 [32000]/60000
loss: 0.928814 [38400]/60000
loss: 0.879323 [44800]/60000

In [49]:
torch.save(model.state_dict(), "model.pth")

In [50]:
model = NeuralNetwork()

In [51]:
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [52]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

In [53]:
model.eval()

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [54]:
x, y = test_data[0][0], test_data[0][1]

In [55]:
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: {predicted}, Actual: {actual}')

Predicted: Ankle boot, Actual: Ankle boot


#### END